In [1]:
from boostsa import Bootstrap

boot = Bootstrap()

boot.test(targs='test_boot/h0.0/targs.txt', h0_preds='test_boot/h0.0/preds.txt', h1_preds='test_boot/h1.0/preds.txt')

bootstrap:  14%|████▏                         | 14/100 [00:00<00:00, 138.39it/s]

total size............... 1000
sample size.............. 100


bootstrap: 100%|█████████████████████████████| 100/100 [00:00<00:00, 135.10it/s]

count sample diff f1   is twice tot diff f1....... 11   / 100     p < 0.11   
count sample diff acc  is twice tot diff acc...... 18   / 100     p < 0.18   
count sample diff prec is twice tot diff prec..... 25   / 100     p < 0.25   
count sample diff rec  is twice tot diff rec ..... 12   / 100     p < 0.12   


,f1,diff_f1,sign_f1,acc,diff_acc,sign_acc,prec,diff_prec,sign_prec,rec,diff_rec,sign_rec
h0,67.76,,,69.0,,,69.94,,,67.96,,
h1,74.07,6.31,,74.1,5.1,,74.10,4.16,,74.22,6.26,


In [2]:
boot = Bootstrap()

boot.feed(h0='h0',          exp_idx='h0.0', preds='test_boot/h0.0/preds.txt', targs='test_boot/h0.0/targs.txt', idxs='test_boot/h0.0/idxs.txt')
boot.feed(h0='h0',          exp_idx='h0.1', preds='test_boot/h0.1/preds.txt', targs='test_boot/h0.1/targs.txt', idxs='test_boot/h0.1/idxs.txt')
boot.feed(h0='h0', h1='h1', exp_idx='h1.0', preds='test_boot/h1.0/preds.txt', targs='test_boot/h1.0/targs.txt', idxs='test_boot/h1.0/idxs.txt')
boot.feed(h0='h0', h1='h1', exp_idx='h1.1', preds='test_boot/h1.1/preds.txt', targs='test_boot/h1.1/targs.txt', idxs='test_boot/h1.1/idxs.txt')
boot.run(n_loops=1000, sample_size=.2, verbose=True)

print("\n\n\n")

next_boot = Bootstrap()
next_boot.loadjson('outcomes.json')

next_boot.feed(h0='h0', h1='h2', exp_idx='h2.0', preds='test_boot/h2.0/preds.txt', targs='test_boot/h2.0/targs.txt', idxs='test_boot/h2.0/idxs.txt')
next_boot.feed(h0='h0', h1='h2', exp_idx='h2.1', preds='test_boot/h2.1/preds.txt', targs='test_boot/h2.1/targs.txt', idxs='test_boot/h2.1/idxs.txt')
next_boot.run(n_loops=1000, sample_size=.2, verbose=True)


bootstrap:   0%|                                       | 0/1000 [00:00<?, ?it/s]

################################################################################
start: 2021/02/09 18:58:27
################################################################################
h0.0                                                         acc 69.0    F 67.76
h0.1                                                         acc 72.6    F 72.59
################################################################################
h0   vs   h1
h1.0                                                         acc 74.1    F 74.07
h1.1                                                         acc 73.0    F 72.99
total size............... 2000
sample size.............. 400
targs count:              ['class 0 freq 930 perc 46.50%', 'class 1 freq 1070 perc 53.50%']
h0 preds count:           ['class 0 freq 892 perc 44.60%', 'class 1 freq 1108 perc 55.40%']
h1 preds count:           ['class 0 freq 1051 perc 52.55%', 'class 1 freq 949 perc 47.45%']
h0 F-measure............. 70.57   h1 F-measure..........

bootstrap:   0%|                                       | 0/1000 [00:00<?, ?it/s]

count sample diff f1   is twice tot diff f1....... 51   / 1000    p < 0.051  
count sample diff acc  is twice tot diff acc...... 59   / 1000    p < 0.059  
count sample diff prec is twice tot diff prec..... 43   / 1000    p < 0.043  *
count sample diff rec  is twice tot diff rec ..... 36   / 1000    p < 0.036  *
   mean_epochs    acc diff_acc sign_acc   prec diff_prec sign_prec    rec diff_rec sign_rec     f1 diff_f1 sign_f1
h0        None  70.80                    70.66                      70.52                    70.57                
h1        None  73.55     2.75           73.79      3.13         *  73.85     3.33        *  73.55    2.98        
################################################################################
end: 2021/02/09 18:58:37  - time elapsed: 00:00:10
################################################################################




################################################################################
start: 2021/02/09 18:58:37
################

bootstrap:   1%|▎                            | 11/1000 [00:00<00:09, 104.60it/s]

count sample diff f1   is twice tot diff f1....... 64   / 1000    p < 0.064  
count sample diff acc  is twice tot diff acc...... 78   / 1000    p < 0.078  
count sample diff prec is twice tot diff prec..... 54   / 1000    p < 0.054  
count sample diff rec  is twice tot diff rec ..... 41   / 1000    p < 0.041  *
################################################################################
h0   vs   h2
h2.0                                                         acc 71.7    F 71.32
h2.1                                                         acc 71.4    F 71.2
total size............... 2000
sample size.............. 400
targs count:              ['class 0 freq 930 perc 46.50%', 'class 1 freq 1070 perc 53.50%']
h0 preds count:           ['class 0 freq 892 perc 44.60%', 'class 1 freq 1108 perc 55.40%']
h2 preds count:           ['class 0 freq 871 perc 43.55%', 'class 1 freq 1129 perc 56.45%']
h0 F-measure............. 70.57   h2 F-measure............. 71.27   diff... 0.7
h0 accuracy....

bootstrap: 100%|███████████████████████████| 1000/1000 [00:08<00:00, 122.24it/s]

count sample diff f1   is twice tot diff f1....... 375  / 1000    p < 0.375  
count sample diff acc  is twice tot diff acc...... 340  / 1000    p < 0.34   
count sample diff prec is twice tot diff prec..... 350  / 1000    p < 0.35   
count sample diff rec  is twice tot diff rec ..... 376  / 1000    p < 0.376  
   mean_epochs    acc diff_acc sign_acc   prec diff_prec sign_prec    rec diff_rec sign_rec     f1 diff_f1 sign_f1
h0        None  70.80                    70.66                      70.52                    70.57                
h1        None  73.55     2.75           73.79      3.13            73.85     3.33        *  73.55    2.98        
h2        None  71.55     0.75           71.46       0.8            71.20     0.68           71.27     0.7        
################################################################################
end: 2021/02/09 18:58:54  - time elapsed: 00:00:16
################################################################################


,mean_epochs,acc,diff_acc,sign_acc,prec,diff_prec,sign_prec,rec,diff_rec,sign_rec,f1,diff_f1,sign_f1
h0,None,70.80,,,70.66,,,70.52,,,70.57,,
h1,None,73.55,2.75,,73.79,3.13,,73.85,3.33,*,73.55,2.98,
h2,None,71.55,0.75,,71.46,0.8,,71.20,0.68,,71.27,0.7,
